# Executing fuzzy SIRDS model to data from the selected municipalities

In [1]:
import pandas as pd
from scipy.optimize import differential_evolution
from datetime import datetime

from model.sirds_model import get_bounds_and_arguments, sirds_objective_function

## Reading data

In [2]:
df = pd.read_csv('../data/df_ts_epidemic_episodes.csv', index_col=0)
df.DATA = pd.to_datetime(df.DATA)

In [3]:
max_date_to_fit = df.DATA.max()
max_date_to_fit

Timestamp('2022-12-31 00:00:00')

In [4]:
cumulative_days_in_first_outbreak_to_max_bound_I0 = 56

## Execution SIRDS model

In [5]:
df = df.sort_values(by=['MUNICIPIO', 'DATA'])

In [6]:
df.columns

Index(['DATA', 'CODIGO_MUNICIPIO_6', 'OBITOS_NOVOS', 'OBITOS', 'day_of_week',
       'OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS', 'MUNICIPIO', 'SIGLA_ESTADO',
       'ESTADO', 'REGIAO', 'POPULACAO_2022', 'TAXA_OBITOS_NOVOS',
       'TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS', 'TAXA_OBITOS',
       'NOVOS_CASOS_SRAG', 'casosNovos', 'casosAcumulado',
       'CASOS_NOVOS_MEDIA_MOVEL_7_DIAS',
       'NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIA',
       'NUMERO_REPRODUCAO_EFETIVO_SRAG_VARIANCIA',
       'NUMERO_REPRODUCAO_EFETIVO_SRAG_QUANTIL_0.025',
       'NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIANA',
       'NUMERO_REPRODUCAO_EFETIVO_SRAG_QUANTIL_0.975',
       'NUMERO_REPRODUCAO_EFETIVO_MEDIA',
       'NUMERO_REPRODUCAO_EFETIVO_VARIANCIA',
       'NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.025',
       'NUMERO_REPRODUCAO_EFETIVO_MEDIANA',
       'NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.975',
       'NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA',
       'NUMERO_REPRODUCAO_EFETIVO_ATRASADO_VARIANCIA',
       'NUMERO_REPRODUCAO_EF

In [10]:
df[df.MUNICIPIO == 'São Paulo'][['DATA','NOVOS_CASOS_SRAG']]

,DATA,NOVOS_CASOS_SRAG
9225,2020-03-12,88.0
9226,2020-03-13,89.0
9227,2020-03-14,136.0
9228,2020-03-15,153.0
9229,2020-03-16,196.0
...,...,...
10245,2022-12-27,25.0
10246,2022-12-28,15.0
10247,2022-12-29,15.0
10248,2022-12-30,12.0


In [7]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS'] = df['CASOS_NOVOS_MEDIA_MOVEL_7_DIAS'] / df['POPULACAO_2022'] * 100000

In [ ]:
try:
    df_initial_results = pd.read_csv('data/execution_results.csv')
except:
    df_initial_results = pd.DataFrame({'municipality_id':[]})

df_results = df_initial_results.copy()

DAYS_TO_RECOVERY = 8

DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR = 5
NUMBER_ESTIMATON_PER_COUNTRY = 20

for estimation in range(NUMBER_ESTIMATON_PER_COUNTRY):        
    for municipality_id in df.CODIGO_MUNICIPIO_6.unique():       
        df_municipality = df[(df.CODIGO_MUNICIPIO_6 == municipality_id) & (df.DATA <= max_date_to_fit)].reset_index(drop=True)
        municipality = df_municipality.MUNICIPIO.unique()[0]
        print('municipality: ', municipality_id, ' ', municipality)
        
        # Dates about begin time series and first case        
        date_first_case = min(df_municipality[(df_municipality.NOVOS_CASOS_SRAG > 0)].iloc[0].DATA, df_municipality[(df_municipality.casosNovos > 0)].iloc[0].DATA)       

        # Period of analysis
        period_in_days = (max_date_to_fit - date_first_case).days + 1
                           
        estimations_performed = len(df_results[(df_results.municipality_id == municipality_id)])
    
        if (estimation == estimations_performed):
            print('estimation: '+str(estimation))
            
            rt_column = 'NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIA'
            # rt_column = 'reproduction_rate'
            
            bounds, args = get_bounds_and_arguments(df_municipality, 'DATA', 'TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS', rt_column, 'TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS', 'ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA', DAYS_TO_RECOVERY, date_first_case, max_date_to_fit, df_municipality.POPULACAO_2022.iloc[0], period_in_days, cumulative_days_in_first_outbreak_to_max_bound_I0)
            
            print(bounds)
            
            # Record the start time
            start_time = datetime.now()
            print(start_time)

            result = differential_evolution(sirds_objective_function, bounds, args=args, popsize=DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR, maxiter=10000, workers=11, updating='deferred')
    
            # Record the end time
            end_time = datetime.now()
    
            # Calculate the duration (in seconds) for the optimization
            duration = (end_time - start_time).total_seconds()
            print(duration)
            
            list_breakpoints_in_slow_transition = args[4]
            quantity_outbreaks = args[5]
            quantity_outbreak_adjustments = args[6]
    
            # Create a dictionary to store results
            estimation_result = {
                'municipality_id': municipality_id,
                'municipality': municipality,
                'estimation': estimation,  # To differentiate between multiple estimations
                'result_fun': result.fun,
                'result_nfev': result.nfev,
                'result_nit': result.nit,
                'result_success': result.success,
                'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format start time as a string
                'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format end time as a string
                'duration_seconds': duration,  # Duration in seconds
                'pop_size': DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR,
                'period_in_days': period_in_days,
                'days_to_recovery': DAYS_TO_RECOVERY,
                'cumulative_days_in_first_outbreak_to_max_bound_I0': cumulative_days_in_first_outbreak_to_max_bound_I0,
                'date_first_case': date_first_case,
                'list_breakpoints_in_slow_transition': list_breakpoints_in_slow_transition,
                'x_initial_infected_population': result.x[0],
                'x_days_between_infections_0': result.x[1]
            }
            
            quantity_epidemic_periods_with_slow_transition = len(list_breakpoints_in_slow_transition) + 1
            
            for p in range(quantity_epidemic_periods_with_slow_transition):
                estimation_result['x_case_fatality_probability_'+str(p)] = result.x[2 + p]
                estimation_result['x_loss_immunity_in_days_'+str(p)] = result.x[2 + quantity_epidemic_periods_with_slow_transition + p]                                                 
            quantity_breakpoints = (quantity_outbreaks - 1) + (quantity_outbreak_adjustments)
            begin_breakpoint_parameters = 2 + 2*quantity_epidemic_periods_with_slow_transition
            for b in range(quantity_breakpoints):
                estimation_result['x_days_between_infections_'+str(b+1)] = result.x[begin_breakpoint_parameters + b]
                estimation_result['x_breakpoint_'+str(b+1)] = result.x[begin_breakpoint_parameters + quantity_breakpoints + b]
                estimation_result['x_transition_days_between_epidemic_periods_'+str(b+1)] = result.x[begin_breakpoint_parameters + 2*quantity_breakpoints + b]
    
            print(estimation_result)
    
            # Append the estimation result to the list            
            df_results = pd.concat([df_results, pd.DataFrame.from_records([estimation_result])])
    
            df_results.to_csv('../data/execution_results.csv', index=False)           

municipality:  520140   Aparecida de Goiânia
municipality:  280030   Aracaju
municipality:  310620   Belo Horizonte
municipality:  150140   Belém
municipality:  350950   Campinas
municipality:  500270   Campo Grande
municipality:  311860   Contagem
municipality:  510340   Cuiabá
municipality:  410690   Curitiba
municipality:  330170   Duque de Caxias
municipality:  291080   Feira de Santana
estimation: 0
[(0.1622631693129248, 693.1067641010351), (5.3182840658122075, 5.934409094895811), (0.0007465742086202607, 0.013300999999999999), (0.0008937709907826997, 0.013300999999999999), (0.00017749058624421722, 0.007678734569747729), (9.900000000000001e-05, 0.0030672771665941523), (89.999999, 365.000001), (89.999999, 365.000001), (89.999999, 365.000001), (89.999999, 365.000001), (5.3182840658122075, 8.557512083272576), (1.9713107732406654, 6.571040244135553), (1.9713107732406654, 8.904070653853912), (2.3530170927474963, 7.843394642491655), (2.3530170927474963, 9.00407328160982), (1.862027758218

In [ ]:
bounds[0]

In [ ]:
df_municipality[['date', 'rate_new_deaths_moving_average', 'rate_new_cases_moving_average', 'estimated_onset_by_deaths', 'NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA']]

In [ ]:
min(df_municipality[(df_municipality.rate_new_cases_moving_average > 0)].iloc[0].date, df_municipality[(df_municipality.estimated_onset_by_deaths > 0)].iloc[0].date)

In [ ]:
df_municipality[(df_municipality.rate_new_cases_moving_average > 0)].iloc[0].date

In [ ]:
df_municipality[(df_municipality.estimated_onset_by_deaths > 0)].iloc[0].date

In [ ]:
outbreaks = df.groupby('ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA').agg({'date': ['min', 'max']})
outbreaks

In [ ]:
outbreaks.iloc[0][('date','min')]

In [ ]:
rt_in_outbreak = df_municipality[
    (df_municipality['date'] >= outbreaks.iloc[0][('date', 'min')]) &
    (df_municipality['date'] <= outbreaks.iloc[0][('date', 'max')])][rt_column].values
rt_in_outbreak

In [ ]:
df_results